https://python.langchain.com/docs/use_cases/sql/quickstart/

### **Test the sqldb**

In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

**Connecting to the sqldb**

In [2]:
db_path = str(here("data")) + "/data_cars_price.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [3]:
db

In [4]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM cars LIMIT 10;")

sqlite
['cars']


"[('BMW', '3 Series', 2018, '330i xDrive', '330i xDrive 4dr Sedan AWD (2.0L 4cyl Turbo 8A)', 'Sedan', 'I4', 'all wheel drive', 'gas', '363.4/521.4 mi.', '$39,795'), ('BMW', '3 Series', 2018, '340i', '340i 4dr Sedan (3.0L 6cyl Turbo 8A)', 'Sedan', 'I6', 'rear wheel drive', 'gas', '331.8/505.6 mi.', '$46,025'), ('BMW', '3 Series', 2018, '340i xDrive', '340i xDrive 4dr Sedan AWD (3.0L 6cyl Turbo 8A)', 'Sedan', 'I6', 'all wheel drive', 'gas', '331.8/489.8 mi.', '$47,885'), ('BMW', '3 Series', 2018, '328d', '328d 4dr Sedan (2.0L 4cyl Turbodiesel 8A)', 'Sedan', 'I4', 'rear wheel drive', 'diesel', '465.0/645.0 mi.', '$39,330'), ('BMW', '3 Series', 2018, '328d xDrive', '328d xDrive 4dr Sedan AWD (2.0L 4cyl Turbodiesel 8A)', 'Sedan', 'I4', 'all wheel drive', 'diesel', '450.0/600.0 mi.', '$41,190'), ('BMW', '3 Series', 2018, '328d xDrive', '328d xDrive 4dr Wagon AWD (2.0L 4cyl Turbodiesel 8A)', 'Wagon', 'I4', 'all wheel drive', 'diesel', '450.0/600.0 mi.', '$43,420'), ('BMW', '3 Series', 2018, '

### **Test the access to the environment variables**

In [5]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: None


In [6]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_GPT_API_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_GPT_ENDPOINT")
)

### **Test your GPT model**

In [7]:
from openai import AzureOpenAI

messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]


response = client.chat.completions.create(
    model=os.getenv("MODEL_GPT_ID"),
    messages=messages
)

print(response.choices[0].message.content)

Hello! How can I assist you today? 😊


In [12]:
# Load the LLM
from langchain.chat_models import AzureChatOpenAI

model_name = os.getenv("MODEL_GPT_ID")
azure_openai_api_key = os.getenv("AZURE_OPENAI_GPT_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_GPT_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"]=os.getenv("AZURE_OPENAI_GPT_API_KEY") 
os.environ["AZURE_OPENAI_ENDPOINT"]=os.getenv("AZURE_OPENAI_GPT_ENDPOINT") 



llm = AzureChatOpenAI(
    openai_api_version="2024-07-01-preview",
    azure_deployment=model_name,
    model_name=model_name,
    temperature=0.0)

### **1. SQL query chain**

In [13]:
# Load the LLM
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version="2024-07-01-preview",
    azure_deployment=model_name,
    model_name=model_name,
    temperature=0.0)

In [15]:
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many cars are there"})
print(response)

```sql
SELECT COUNT(*) AS "Total_Cars" FROM cars;
```


In [21]:
from langchain.chains import create_sql_query_chain
import re

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many cars are there"})


if isinstance(response, dict):
    sql_query = response.get("query") or response.get("sql") or ""
else:
    sql_query = response

# Remove Markdown code block formatting (e.g., ```sql ... ```)
sql_query_clean = re.sub(r"```(?:\w+)?\n", "", sql_query).strip()
sql_query_clean = sql_query_clean.rstrip("```").strip()

print("Clean SQL Query:", sql_query_clean)


Clean SQL Query: SELECT COUNT(*) AS "Total_Cars" FROM cars;


In [ ]:
import re
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain

# Define a function to remove Markdown formatting from the SQL query.
def remove_markdown(sql_text: str) -> str:
    # Remove the starting triple backticks and any language specifier (like "sql")
    cleaned = re.sub(r"```(?:sql)?\n", "", sql_text)
    # Remove any trailing triple backticks
    cleaned = re.sub(r"```", "", cleaned)
    return cleaned.strip()

# Create the query-writing chain.
write_query = create_sql_query_chain(llm, db)

# Invoke the chain to generate the SQL query.
raw_response = write_query.invoke({"question": "How many cars are there"})

# Depending on the chain's output format, extract the SQL query.
# For example, if it's a string:
if isinstance(raw_response, str):
    raw_query = raw_response
elif isinstance(raw_response, dict):
    # Adjust the key name as needed.
    raw_query = raw_response.get("query", "")
else:
    raw_query = str(raw_response)

# Clean the SQL query by removing Markdown formatting.
clean_query = remove_markdown(raw_query)

# Now execute the clean query.
execute_query = QuerySQLDataBaseTool(db=db)
result = execute_query.invoke({"query": clean_query})

print("Generated SQL Query:")
print(clean_query)
print("Execution Result:")
print(result)


Execute the query to make sure it’s valid

In [22]:
db.run(sql_query_clean)

'[(214,)]'

In [23]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### **Add QuerySQLDataBaseTool to the chain**
Execute SQL query

**This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [24]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "How many cars are there"})

'Error: (sqlite3.OperationalError) near "```sql\nSELECT COUNT(*) AS "Total_Cars" FROM cars;\n```": syntax error\n[SQL: ```sql\nSELECT COUNT(*) AS "Total_Cars" FROM cars;\n```]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [26]:
import re
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain

# Define a function to remove Markdown formatting from the SQL query.
def remove_markdown(sql_text: str) -> str:
    # Remove the starting triple backticks and any language specifier (like "sql")
    cleaned = re.sub(r"```(?:sql)?\n", "", sql_text)
    # Remove any trailing triple backticks
    cleaned = re.sub(r"```", "", cleaned)
    return cleaned.strip()

# Create the query-writing chain.
write_query = create_sql_query_chain(llm, db)

# Invoke the chain to generate the SQL query.
raw_response = write_query.invoke({"question": "How many cars are there"})

# Depending on the chain's output format, extract the SQL query.
# For example, if it's a string:
if isinstance(raw_response, str):
    raw_query = raw_response
elif isinstance(raw_response, dict):
    # Adjust the key name as needed.
    raw_query = raw_response.get("query", "")
else:
    raw_query = str(raw_response)

# Clean the SQL query by removing Markdown formatting.
clean_query = remove_markdown(raw_query)

# Now execute the clean query.
execute_query = QuerySQLDataBaseTool(db=db)
result = execute_query.invoke({"query": clean_query})

print("Generated SQL Query:")
print(clean_query)
print("Execution Result:")
print(result)


Generated SQL Query:
SELECT COUNT(*) AS "Total_Cars" FROM cars;
Execution Result:
[(214,)]


### **Answer the question in a user friendly manner**

In [27]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many cars are there"})

'The SQL query provided contains syntax errors due to the inclusion of unnecessary formatting markers (```sql). These markers are not valid SQL syntax and are causing the error. \n\nTo fix the issue, the query should be written as:\n\n```sql\nSELECT COUNT(*) AS "Total_Cars" FROM cars;\n```\n\nIf the corrected query is executed successfully, it will return the total number of cars in the `cars` table. However, since the query did not execute due to the error, the number of cars cannot be determined from the provided information. \n\nTo answer the user\'s question, the query needs to be corrected and re-executed.'

In [29]:
import re
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Define a function to remove markdown formatting from SQL query text.
def remove_markdown(sql_text: str) -> str:
    # Remove starting triple backticks and optional "sql" language tag.
    cleaned = re.sub(r"```(?:sql)?\n", "", sql_text)
    # Remove any trailing triple backticks.
    cleaned = re.sub(r"```", "", cleaned)
    return cleaned.strip()

# Create a runnable version of the markdown removal function.
remove_markdown_runnable = RunnableLambda(remove_markdown)

# Your answer prompt remains the same.
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

# Build the answer chain.
answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=write_query | remove_markdown_runnable)
    .assign(result=itemgetter("query") | execute_query)
    | answer
)

# Invoke the chain.
final_response = chain.invoke({"question": "How many cars are there"})
print(final_response)


There are 214 cars.


### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions
- …

In [13]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [14]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Invoice,Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José 

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The country whose customers spent the most is the USA. The total sales from customers in the USA is $523.06.'}

In [15]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})
# agent_executor.invoke("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/The `PlaylistTrack` table has two columns: `PlaylistId` and `TrackId`. It is a junction table that represents the relationship between playlists and tracks. The `PlaylistId` column is a foreign key referencing the `Playlist` table, and the `TrackId` column is a foreign key referencing the `Track` table. The primary key of the table is a combination of both columns.

Here are three sample rows fro

{'input': 'Describe the playlisttrack table',
 'output': 'The `PlaylistTrack` table has two columns: `PlaylistId` and `TrackId`. It is a junction table that represents the relationship between playlists and tracks. The `PlaylistId` column is a foreign key referencing the `Playlist` table, and the `TrackId` column is a foreign key referencing the `Track` table. The primary key of the table is a combination of both columns.\n\nHere are three sample rows from the `PlaylistTrack` table:\n\n| PlaylistId | TrackId |\n|------------|---------|\n| 1          | 3402    |\n| 1          | 3389    |\n| 1          | 3390    |'}